In [1]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, boxcox, pearsonr, ks_2samp
import matplotlib.pyplot as plt

import json
import os
os.chdir("..")

def metric_train(output, truth):
    return spearmanr(output, truth).correlation

def join_preds(strat_models_dict:dict, filename:str="test_preds.csv"):
    ldf = None
    for i, (k, v) in enumerate(strat_models_dict.items()):
        df = pd.read_csv(f"models/{k}/{filename}", index_col="ID", usecols=["ID"] + v)
        if i == 0:
            ldf = df
        else:
            ldf = ldf.join(df, rsuffix=f"_{k}")
    return ldf

In [2]:
STRATEGY_MODELS = {
    "lasso_selected_features_linear_models": ["lasso", "ridge"],
    "voting_models_all_feat": ["xgb", "lgbm", "rf"],
}

predictions = join_preds(STRATEGY_MODELS, filename="test_pred_bootstrap.csv")
predictions

,lasso,ridge,lgbm,rf,xgb
ID,,,,,
730,334.725291,363.949231,353.297957,522.848777,0.554728
273,506.066366,450.265222,553.298903,641.350643,0.571674
911,343.156737,381.033586,288.985486,521.843304,0.555494
274,853.662328,872.141250,864.534730,737.917803,0.598850
619,724.613424,720.097786,701.912765,790.760161,0.600582
...,...,...,...,...,...
1155,670.111311,670.764176,632.417451,685.641111,405.745700
1114,689.246890,702.275635,772.741280,762.678316,462.495640
939,703.654789,691.936612,784.355634,719.821807,453.433500


In [3]:
sub_dummy = pd.read_csv("submissions/y_test_random_final.csv", index_col="ID", usecols=["ID"])

test_pred = predictions.rank().mean(axis=1).rename("TARGET")
test_pred = sub_dummy.join(test_pred)
# test_pred.to_csv("submissions/sub30_ridge_lasso_selected_tree_all_bootstrap.csv")

test_pred

,TARGET
ID,
1115,319.4
1202,416.6
1194,176.0
1084,478.0
1135,184.4
...,...
879,20.2
673,204.6
1641,586.2


In [4]:
# Comparing the earlier submission
ref_sub_num = "sub28"
ref_filename = [f for f in os.listdir("submissions") if ref_sub_num in f][0]
ref_sub = pd.read_csv(f"submissions/{ref_filename}")
metric_train(ref_sub["TARGET"], test_pred["TARGET"])

0.9855302573295771